This is a complex SQL query that aggregates television viewing data from a source table, ext_data_stream.inscape_content_data, and classifies households based on their viewing patterns. The query has multiple Common Table Expressions (CTEs) to break down the problem into smaller steps. Here's a breakdown of the query:

1. The first CTE named content aggregates the source data by year, month, week number, day, household, and TV ID. It calculates the total content duration in hours for each household and the number of events (or occurrences of content playback).
2. The second CTE named temp_table uses the content CTE to calculate percentiles for the total content duration for households across weeks. The percentiles are used later to classify households based on their viewing patterns.
3. The third CTE named viewership_table joins the temp_table and content CTEs and calculates the household viewing classification based on the total content duration, distinct TV count, and percentiles from temp_table.

Overall, the query performs a series of aggregations on the source data to classify households based on their viewing patterns. It calculates the total content duration, distinct TV count, and other metrics at various levels of granularity, including year, month, week, and household. Finally, it uses percentiles to assign households to one of four classes: light, medium, heavy, and heavy plus.
    

In [ ]:
WITH content AS (
    SELECT
        year,
        month,
        week_of_year(CAST(CONCAT(year, '-', month, '-', day) AS DATE)) AS week_num,
        day,
        ip AS household_id,
        tv_id,
        SUM(
            CASE
                WHEN (date_diff('minute', cont_recog_start_time, cont_recog_end_time))/60.0 = 0 THEN 1
                ELSE (date_diff('minute', cont_recog_start_time, cont_recog_end_time))/60.0
            END
        ) AS cont_dur_hr,
        COUNT() * 0.5 AS events
    FROM ext_data_stream.inscape_content_data
    WHERE year = '2022'
        -- AND month  = '08'
        -- AND day = '01'
        AND week_of_year(CAST(CONCAT(year, '-', month, '-', day) AS DATE)) IN (31) 
    GROUP BY 1, 2, 3, 4, 5, 6
), temp_table AS (
    SELECT
        w.year,
        w.month,
        w.week_num,
        APPROX_PERCENTILE(ROUND(w.median_hr, 2), 0.3) AS light,
        APPROX_PERCENTILE(ROUND(w.median_hr, 2), 0.6) AS medium,
        APPROX_PERCENTILE(ROUND(w.median_hr, 2), 0.9) AS heavy,
        APPROX_PERCENTILE(ROUND(w.median_hr, 2), 1.0) AS heavy_plus
    FROM (
        SELECT
            d.year,
            d.month,
            d.week_num,
            d.household_id,
            COUNT(DISTINCT d.day) AS distinct_days,
            COUNT(DISTINCT d.tv_id) AS distinct_tvs,
            SUM(d.cont_dur_hr) AS total_dur_hr,
            APPROX_PERCENTILE(d.cont_dur_hr, 0.5) AS median_hr,
            APPROX_PERCENTILE(d.events, 0.5) AS median_events,
            AVG(d.cont_dur_hr) AS avg_dur_hr
        FROM (SELECT * FROM content) d
        GROUP BY 1, 2, 3, 4
    ) w
    GROUP BY 1, 2, 3),

WITH viewership_table AS (
    SELECT
        w.year, w.month, w.week_num,
        CASE
            WHEN ROUND(avg_dur_hr, 2) > t.heavy AND ROUND(avg_dur_hr, 2)/distinct_tvs <= 1.7 THEN 'light'
            WHEN ROUND(avg_dur_hr, 2) > t.heavy AND ROUND(avg_dur_hr, 2)/distinct_tvs > 1.7 AND ROUND(avg_dur_hr, 2)/distinct_tvs <= 5 THEN 'medium'
            WHEN ROUND(avg_dur_hr, 2) <= t.light THEN 'light'
            WHEN ROUND(avg_dur_hr, 2) > t.light AND ROUND(avg_dur_hr, 2) <= t.medium THEN 'medium'
            WHEN ROUND(avg_dur_hr, 2) > t.medium AND ROUND(avg_dur_hr, 2) <= t.heavy THEN 'heavy'
            ELSE 'heavy_plus'
        END AS viewership,
        COALESCE(
            CASE
                WHEN ROUND(avg_dur_hr, 2) > t.heavy AND ROUND(avg_dur_hr, 2)/distinct_tvs <= 1.7 THEN ROUND(t.light, 2)
                WHEN ROUND(avg_dur_hr, 2) > t.heavy AND ROUND(avg_dur_hr, 2)/distinct_tvs > 1.7 AND ROUND(avg_dur_hr, 2)/distinct_tvs <= 5 THEN ROUND(t.medium, 2)
                WHEN ROUND(avg_dur_hr, 2) <= t.light THEN ROUND(t.light, 2)
                WHEN ROUND(avg_dur_hr, 2) > t.light AND ROUND(avg_dur_hr, 2) <= t.medium THEN ROUND(t.medium, 2)
                WHEN ROUND(avg_dur_hr, 2) > t.medium AND ROUND(avg_dur_hr, 2) <= t.heavy THEN ROUND(t.heavy, 2)
                ELSE ROUND(t.heavy_plus, 2)
            END
        ) AS max_hrs,
        COUNT(*) AS household_id_count,
        COUNT(household_id) * 100.0 / SUM(COUNT(household_id)) OVER (PARTITION BY w.year, w.month, w.week_num ORDER BY w.week_num ASC) AS pct_total
    FROM (
        SELECT
            d.year, d.month, d.week_num,
            d.household_id,
            AVG(d.cont_dur_hr) AS avg_dur_hr,
            APPROX_PERCENTILE(d.cont_dur_hr, 0.5) AS median_hr,
            SUM(d.distinct_tvs) AS distinct_tvs,
            COUNT(DISTINCT d.day) AS distinct_days,
            APPROX_PERCENTILE(distinct_tvs, 0.5) AS median_tvs,
            COUNT(*) AS cnt
        FROM (
            SELECT
                household_id,
                year,
                month,
                day,
                WEEK_OF_YEAR(CAST(CONCAT(year, '-', month, '-', day) AS DATE)) AS week_num,
                SUM(cont_dur_hr) AS cont_dur_hr,
                COUNT(DISTINCT tv_id) AS distinct_tvs,
                COUNT(DISTINCT day) AS distinct_days
            FROM content
            GROUP BY 1, 2, 3, 4, 5
        ) d
        GROUP BY 1, 2, 3, 4
    ) w
    LEFT JOIN temp_table t ON w.year = t.year AND w.month = t.month AND w.week_num = t.week_num
    GROUP BY 1, 2, 3, 4, 5
),

,final_table as (
select 
    viewership, household_id_count, 
    round(pct_total,2) as pct_total, max_hrs,
    viewership_table.year,
    viewership_table.month, 
    viewership_table.week_num
from viewership_table 
)
select * from final_table;